In [1]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import random
from keras.models import load_model, Sequential, Model
from keras.layers import Cropping2D
import cv2


Using TensorFlow backend.


In [2]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
import cv2, numpy as np

def Model(weights_path=None, dropout=0.5, dropout_level=1, orig = True):
    if orig:
        model = Sequential()
        
        # Take 2x2 stride on the input to reduce dimensionality
        model.add(Lambda(lambda x: x[::2, ::2, :], input_shape=(3, 160, 320), output_shape=(3, 80, 160))) #(3, 80, 160)
        
        # Zero pad the width
        model.add(ZeroPadding2D(padding=(0, 20))) #(3, 80, 200)
        
        # Crop the height
        model.add(Cropping2D(cropping=((80-66, 0), (0, 0)))) #(3, 66, 200)
        
        # Normalize
        model.add(Lambda(lambda x: (x / 255.0) - 0.5, output_shape=(3, 66, 200)))
        
        # Original network
        model.add(Dropout(dropout)) if dropout_level >= 2 else None
        model.add(Convolution2D(24, 5, 5, border_mode='valid', subsample=(2,2), activation='relu')) #(24, 31, 98)
        model.add(Dropout(dropout)) if dropout_level >= 2 else None
        model.add(Convolution2D(36, 5, 5, border_mode='valid', subsample=(2,2), activation='relu')) #(36, 14, 47)
        model.add(Dropout(dropout)) if dropout_level >= 2 else None
        model.add(Convolution2D(48, 5, 5, border_mode='valid', subsample=(2,2), activation='relu')) #(48, 5, 22)
        model.add(Dropout(dropout)) if dropout_level >= 2 else None
        model.add(Convolution2D(64, 3, 3, border_mode='valid', subsample=(1,1), activation='relu')) #(64, 3, 20)
        model.add(Dropout(dropout)) if dropout_level >= 2 else None
        model.add(Convolution2D(64, 3, 3, border_mode='valid', subsample=(1,1), activation='relu')) #(64, 1, 18)
        model.add(Dropout(dropout)) if dropout_level >= 2 else None
        model.add(Flatten())
        model.add(Dense(100, activation='relu'))
        model.add(Dropout(dropout)) if dropout_level >= 1 else None
        model.add(Dense(50, activation='relu'))
        model.add(Dropout(dropout)) if dropout_level >= 1 else None
        model.add(Dense(10, activation='relu'))
        model.add(Dropout(dropout)) if dropout_level >= 1 else None
        model.add(Dense(1, activation='tanh'))
    else:
        model = Sequential()
        model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(3, 160, 320), output_shape=(3, 160, 320)))
        model.add(Dropout(dropout)) if dropout_level >= 2 else None
        model.add(Convolution2D(24, 5, 5, border_mode='valid', subsample=(3,3), activation='relu')) #(24, 52, 106)
        model.add(Dropout(dropout)) if dropout_level >= 2 else None
        model.add(Convolution2D(36, 5, 5, border_mode='valid', subsample=(2,2), activation='relu')) #(36, 24, 51)
        model.add(Dropout(dropout)) if dropout_level >= 2 else None
        model.add(Convolution2D(48, 5, 5, border_mode='valid', subsample=(2,2), activation='relu')) #(48, 10, 24)
        model.add(Dropout(dropout)) if dropout_level >= 2 else None
        model.add(Convolution2D(64, 3, 3, border_mode='valid', subsample=(1,1), activation='relu')) #(64, 8, 22)
        model.add(Dropout(dropout)) if dropout_level >= 2 else None
        model.add(Convolution2D(64, 3, 3, border_mode='valid', subsample=(1,1), activation='relu')) #(64, 6, 20)
        model.add(Dropout(dropout)) if dropout_level >= 2 else None
        model.add(Flatten())
        model.add(Dense(100, activation='relu'))
        model.add(Dropout(dropout)) if dropout_level >= 1 else None
        model.add(Dense(50, activation='relu'))
        model.add(Dropout(dropout)) if dropout_level >= 1 else None
        model.add(Dense(10, activation='relu'))
        model.add(Dropout(dropout)) if dropout_level >= 1 else None
        model.add(Dense(1, activation='tanh'))

    if weights_path:
        model.load_weights(weights_path, by_name=True)

    return model

In [3]:
data_dir = "data_download"

driving_log_df = pd.read_csv(data_dir + "/driving_log.csv")
driving_log_df[['center', 'left', 'right']] = data_dir + "/" + driving_log_df[['center', 'left', 'right']]

# shuffle
np.random.seed(42)
driving_log_df = driving_log_df.reindex(np.random.permutation(driving_log_df.index), copy=False)

driving_log_df.head()

,center,left,right,steering,throttle,brake,speed
6637,data_download/IMG/center_2016_12_01_13_44_19_9...,data_download/ IMG/left_2016_12_01_13_44_19_95...,data_download/ IMG/right_2016_12_01_13_44_19_9...,-0.097735,0.985533,0.0,30.18678
6632,data_download/IMG/center_2016_12_01_13_44_19_4...,data_download/ IMG/left_2016_12_01_13_44_19_44...,data_download/ IMG/right_2016_12_01_13_44_19_4...,-0.078746,0.985533,0.0,30.18562
7327,data_download/IMG/center_2016_12_01_13_45_29_9...,data_download/ IMG/left_2016_12_01_13_45_29_92...,data_download/ IMG/right_2016_12_01_13_45_29_9...,-0.002791,0.985533,0.0,30.18650
730,data_download/IMG/center_2016_12_01_13_33_52_2...,data_download/ IMG/left_2016_12_01_13_33_52_24...,data_download/ IMG/right_2016_12_01_13_33_52_2...,0.000000,0.985533,0.0,30.18665
1515,data_download/IMG/center_2016_12_01_13_35_11_8...,data_download/ IMG/left_2016_12_01_13_35_11_81...,data_download/ IMG/right_2016_12_01_13_35_11_8...,0.000000,0.985533,0.0,30.18685


In [4]:
def get_next_image_generator(df, position = 'center', offset = 0.2):
    for idx, image_path in enumerate(df[position]):
        img = cv2.imread(image_path)
        yield img


In [5]:
tempgen = get_next_image_generator(driving_log_df)
sample = next(tempgen)
print("Dimension of image: H x W X D = ", sample.shape)
print("# of images: ", len(driving_log_df))

print("Steering range: Min=", np.min(driving_log_df['steering']), " , Max=", np.max(driving_log_df['steering']))
print("Throttle range: Min=", np.min(driving_log_df['throttle']), " , Max=", np.max(driving_log_df['throttle']))
print("Brake range: Min=", np.min(driving_log_df['brake']), " , Max=", np.max(driving_log_df['brake']))
print("Speed range: Min=", np.min(driving_log_df['speed']), " , Max=", np.max(driving_log_df['speed']))

print("image Min: ", np.min(sample))
print("image Max: ", np.max(sample))
sample

Dimension of image: H x W X D =  (160, 320, 3)
# of images:  8036
Steering range: Min= -0.9426954  , Max= 1.0
Throttle range: Min= 0.0  , Max= 0.9855326
Brake range: Min= 0.0  , Max= 1.0
Speed range: Min= 0.5024896  , Max= 30.70936
image Min:  0
image Max:  255


array([[[187, 143, 130],
        [185, 141, 128],
        [181, 137, 124],
        ..., 
        [ 14,  38,  30],
        [ 14,  35,  26],
        [  1,  20,  11]],

       [[162, 129, 114],
        [166, 133, 118],
        [171, 138, 123],
        ..., 
        [  0,  15,   6],
        [  0,   9,   1],
        [ 11,  34,  26]],

       [[115,  99,  83],
        [122, 106,  90],
        [134, 118, 102],
        ..., 
        [154, 184, 179],
        [  0,  11,   7],
        [  0,   6,   0]],

       ..., 
       [[ 91, 105, 104],
        [ 97, 111, 110],
        [101, 115, 114],
        ..., 
        [ 82,  96,  95],
        [ 75,  89,  88],
        [ 78,  92,  91]],

       [[110, 124, 123],
        [ 99, 113, 112],
        [ 87, 101, 100],
        ..., 
        [ 90, 104, 103],
        [ 85,  99,  98],
        [ 87, 101, 100]],

       [[ 88, 102, 101],
        [ 86, 100,  99],
        [ 89, 103, 102],
        ..., 
        [ 99, 113, 112],
        [101, 115, 114],
        [ 91, 105,

In [6]:
def get_next_feature(df, batch_size = 10, mode = 'train', position = 'center', offset = 0.2, val_portion = 0.2):
    """
    Assumes df is already shuffled
    """
    total_len = len(df)
    val_len = int(val_portion * total_len)
    train_len = total_len - val_len
    if mode == "train":
        df = df[:train_len]
    else:
        df = df[train_len:]
            
    sample_image = cv2.imread(df['center'].iloc[0])
    sample_image = np.rollaxis(sample_image, 2, 0)
    image_size = sample_image.shape
    
    inputs = np.zeros([batch_size, *image_size]) #length of prediction output
    targets = np.zeros([batch_size])
    
    count = 0
    
    while(True):
        for idx in range(len(df)):
            image_path = df[position].iloc[idx]
            img = cv2.imread(image_path)
            img = img[np.newaxis, :, :, :]
            img = np.rollaxis(img, 3, 1)        

            inputs[count] = img
            targets[count] = df.iloc[idx]['steering']
            if position == 'right':
                targets[count] -= offset
            elif position == 'left':
                targets[count] += offset

            count += 1
            if count == batch_size:
                yield inputs, targets
                inputs = np.zeros([batch_size, *image_size])
                targets = np.zeros([batch_size])
                count = 0


In [7]:
# Define the model

#model = Model() # The nVidia model
model = Model(dropout=0.5, dropout_level=0, orig = False)

ValueError: Negative dimension size caused by subtracting 5 from 3 for 'Conv2D' (op: 'Conv2D') with input shapes: [?,3,160,320], [5,5,320,24].

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=["accuracy"])

# train model
EPOCHS = 5 #5
BATCH_SIZE = 300
OFFSET = 0.2
VAL_PORTION = 0.2

# Train on the center data
position = 'center'
train_generator_center = get_next_feature(driving_log_df, 10, 'train', position, OFFSET, VAL_PORTION)
validation_generator_center = get_next_feature(driving_log_df, 10, 'val', position, OFFSET, VAL_PORTION)
model.fit_generator(train_generator_center, BATCH_SIZE, EPOCHS, verbose=2, validation_data=validation_generator_center, nb_val_samples=BATCH_SIZE/3)

# Train on the left camera data
position = 'left'
train_generator_left = get_next_feature(driving_log_df, 10, 'train', position, OFFSET, VAL_PORTION)
validation_generator_left = get_next_feature(driving_log_df, 10, 'val', position, OFFSET, VAL_PORTION)
model.fit_generator(train_generator_left, BATCH_SIZE, EPOCHS, verbose=2, validation_data=validation_generator_left, nb_val_samples=BATCH_SIZE/3)

# Train on the right camera data
position = 'right'
train_generator_right = get_next_feature(driving_log_df, 10, 'train', position, OFFSET, VAL_PORTION)
validation_generator_right = get_next_feature(driving_log_df, 10, 'val', position, OFFSET, VAL_PORTION)
model.fit_generator(train_generator_right, BATCH_SIZE, EPOCHS, verbose=2, validation_data=validation_generator_right, nb_val_samples=BATCH_SIZE/3)


In [ ]:
model.save('model.h5')

# # returns a compiled model
# # identical to the previous one
# model = load_model('model.h5')